In [ ]:
!pip install yfinance
!pip install ta

In [ ]:
import yfinance as yf
from ta import add_all_ta_features
import pandas as pd
import numpy as np



In [ ]:
# df = pd.read_csv('GGAL (1).csv')
ggal = yf.Ticker("ggal")
df = ggal.history(interval='1d', start='2001-01-01', end='2024-02-23')
# df.to_csv('ggal.csv')
# df = pd.read_csv('ggal.csv')

In [ ]:
# Calcular todas las características técnicas
df = add_all_ta_features(df, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)

df["Tomorrow"] = df["Close"].shift(-1)

df["Target"] = (df["Tomorrow"] > df["Close"]).astype(int)

df = df.drop(['Dividends', 'Stock Splits'], axis = 1 )

df.index = df.index.tz_localize(None)

df = df.dropna()

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, shuffle=False)

In [ ]:
predictors = ['Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'volume_adi',
 'volume_obv',
 'volume_cmf',
 'volume_fi',
 'volume_em',
 'volume_sma_em',
 'volume_vpt',
 'volume_vwap',
 'volume_mfi',
 'volume_nvi',
 'volatility_bbm',
 'volatility_bbh',
 'volatility_bbl',
 'volatility_bbw',
 'volatility_bbp',
 'volatility_bbhi',
 'volatility_bbli',
 'volatility_kcc',
 'volatility_kch',
 'volatility_kcl',
 'volatility_kcw',
 'volatility_kcp',
 'volatility_kchi',
 'volatility_kcli',
 'volatility_dcl',
 'volatility_dch',
 'volatility_dcm',
 'volatility_dcw',
 'volatility_dcp',
 'volatility_atr',
 'volatility_ui',
 'trend_macd',
 'trend_macd_signal',
 'trend_macd_diff',
 'trend_sma_fast',
 'trend_sma_slow',
 'trend_ema_fast',
 'trend_ema_slow',
 'trend_vortex_ind_pos',
 'trend_vortex_ind_neg',
 'trend_vortex_ind_diff',
 'trend_trix',
 'trend_mass_index',
 'trend_dpo',
 'trend_kst',
 'trend_kst_sig',
 'trend_kst_diff',
 'trend_ichimoku_conv',
 'trend_ichimoku_base',
 'trend_ichimoku_a',
 'trend_ichimoku_b',
 'trend_stc',
 'trend_adx',
 'trend_adx_pos',
 'trend_adx_neg',
 'trend_cci',
 'trend_visual_ichimoku_a',
 'trend_visual_ichimoku_b',
 'trend_aroon_up',
 'trend_aroon_down',
 'trend_aroon_ind',
 'trend_psar_up',
 'trend_psar_down',
 'trend_psar_up_indicator',
 'trend_psar_down_indicator',
 'momentum_rsi',
 'momentum_stoch_rsi',
 'momentum_stoch_rsi_k',
 'momentum_stoch_rsi_d',
 'momentum_tsi',
 'momentum_uo',
 'momentum_stoch',
 'momentum_stoch_signal',
 'momentum_wr',
 'momentum_ao',
 'momentum_roc',
 'momentum_ppo',
 'momentum_ppo_signal',
 'momentum_ppo_hist',
 'momentum_pvo',
 'momentum_pvo_signal',
 'momentum_pvo_hist',
 'momentum_kama',
 'others_dr',
 'others_dlr',
 'others_cr']

In [ ]:
X_train = train[predictors]
y_train = train["Target"]

X_test = test[predictors]
y_test = test["Target"]

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Supongamos que X_train y X_test son tus conjuntos de datos de características

# Crea el objeto StandardScaler
scaler = StandardScaler()

# Ajusta el scaler en el conjunto de entrenamiento y transforma los datos
X_train_scaled = scaler.fit_transform(X_train)

# Aplica la misma transformación al conjunto de prueba
X_test_scaled = scaler.transform(X_test)

In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# # Escalar los conjuntos de entrenamiento y prueba utilizando Min-Max Scaling
# scaler = MinMaxScaler()
# X_train_scaled = scaler.fit_transform(X_train)

# X_test_scaled = scaler.transform(X_test)

# # Resto del código para la creación y entrenamiento del modelo...

In [ ]:
def create_sequences_with_shifted_target(data, target, window_size):
    sequences = []
    labels = []

    for i in range(len(data) - window_size):
        sequence = data[i:i+window_size]
        label = target[i+window_size]
        sequences.append(sequence)
        labels.append(label)

    return np.array(sequences), np.array(labels)

In [ ]:
window_size = 50

In [ ]:
X_train_sequences, y_train_sequences = create_sequences_with_shifted_target(X_train_scaled, y_train, window_size)

In [ ]:
X_test_sequences, y_test_sequences = create_sequences_with_shifted_target(X_test_scaled, y_test, window_size)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
# Crea el objeto Sequential y agrega capas

drop_out = 0.2
optimizador = Adam(learning_rate=1e-4)

model = Sequential()
model.add(LSTM(256, activation='relu', input_shape=(window_size, X_train_scaled.shape[1]), return_sequences=False))
# model.add(Dropout(drop_out))
# model.add(LSTM(256, activation='relu', return_sequences=True))
# model.add(Dropout(drop_out))
# model.add(LSTM(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# model = Sequential()
# model.add(LSTM(256, activation='relu', input_shape=(X_train_scaled.shape[1], 1), return_sequences=True))
# model.add(Dropout(drop_out))  # Capa de dropout para prevenir sobreajuste
# model.add(LSTM(256, activation='relu', return_sequences=True))  # Segunda capa LSTM
# # model.add(Dropout(drop_out))  # Capa de dropout adicional
# # model.add(LSTM(64, activation='relu', return_sequences=True))
# model.add(Dense(1, activation='sigmoid'))

# Compila el modelo

model.compile(optimizer=optimizador, loss='binary_crossentropy', metrics=['binary_accuracy'])


In [ ]:
model.fit(X_train_sequences, y_train_sequences, epochs=100, validation_split=0.2)